# RNN계열 모델로 뉴스 기사 분류하기

- 지금까지 배운 다양한 모델을 활용하여 가장 성능 좋은 모델을 찾는 과제 입니다. 
- 1~2일차 실습자료의 DNN, SimpleRNN, GRU, LSTM 모델과 4일차 Bi-LSTM, RNN 등을 활용하여 뉴스기사를 분류합니다.
- tensorflow의 공식 홈페이지 링크를 참고하여, 다양한 모델 형태를 생각해볼 수 있습니다. 
  - [Simple Neural Network](https://www.tensorflow.org/tutorials/keras/text_classification#%EB%AA%A8%EB%8D%B8_%EA%B5%AC%EC%84%B1)
  - [RNN](https://www.tensorflow.org/tutorials/text/text_classification_rnn#create_the_model)

In [1]:
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/PBL_0115'

Mounted at /content/drive


## Train Set과 Test Set 생성하기

- `news_classification_train.csv`과 `news_classification_test.csv` 불러오기
- 각각을 활용하여 X_train과 X_test를 만든다
  - `RNN-classification` 실습자료 참고
- `X_train`과 `X_test`를 만들때, ''이 없도록 제거한다. filter 등을 사용할 수 있다
  - 예시) [['아', ' ', '더', '빙', '진짜', '짜증', '나', '네요', '목소리']] -> ' ' 항목은 제거한다

In [33]:
import pandas as pd
import os

In [34]:
train_df = pd.read_csv(os.path.join(path, 'news_classification_train.csv'), sep='\t', encoding='utf-8', index_col=0)
test_df = pd.read_csv(os.path.join(path, 'news_classification_test.csv'), sep='\t', encoding='utf-8', index_col=0)

In [35]:
train_df.head()

,preprocessed_sent,label_encoded
52,공공 SW사업 불공정 하도급 관행 개선되나 공공 소프트웨어 SW 사업 수주시 제안서...,0
3529,밀레 고탄성 소재로 발이 편안 엠리밋 천연 방충성분 넣은 봄재킷 이제 아웃도어...,3
6016,경마코너 새해 고객 환급률 73 로 2009년 과천 서울경마공원에서 펼쳐지는 경...,6
8046,도의회 예결특위 선정 잡음 경기도의회가 7대 마지막 예산결산특별위원회 위원 선정...,8
1109,한국지엠 부품 협력업체 상생 강화 경영 현황 설명회 370여명 참석 카젬 사...,1


In [36]:
X_train = [ list(filter(None, x.split(' ')))  for x in train_df['preprocessed_sent'].values]
X_test = [ list(filter(None, x.split(' ')))  for x in test_df['preprocessed_sent'].values]

In [37]:
X_train[:1]

[['공공',
  'SW사업',
  '불공정',
  '하도급',
  '관행',
  '개선되나',
  '공공',
  '소프트웨어',
  'SW',
  '사업',
  '수주시',
  '제안서',
  '작성에',
  '협력한',
  '중소기업을',
  '사업수주',
  '후',
  '일방적으로',
  '변경하는',
  '관행',
  '등',
  'SW사업',
  '하도급',
  '단계별로',
  '겪는',
  '애로가',
  '상당',
  '부분',
  '해소될',
  '것으로',
  '보인다',
  '지식경제부는',
  '8일',
  '이명박',
  '대통령',
  '주재로',
  '열린',
  '제76차',
  '국민경제대책회의에서',
  '불공정',
  '하도급',
  '관행',
  '개선을',
  '위한',
  '공공',
  'SW',
  '사업관련',
  '제도개선',
  '방안을',
  '발표했다']]

## Additional preprocessing

- `RNN-classification` 실습자료를 참고합니다.
- Tokenizer 구성하고, max_len를 활용하여 pad_sequences를 적용합니다.
- Multi-classification이기 때문에 y_test, y_train은 onehot-encoding으로 바꿔줘야 합니다.
  - 아래와 같은 형태로 출력이 되야 합니다. [참고 링크](https://m.blog.naver.com/PostView.nhn?blogId=wideeyed&logNo=221343373342&proxyReferer=https:%2F%2Fwww.google.com%2F)
  ```
  array([[1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 1., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])
  ```



In [38]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [39]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [40]:
def below_threshold_len(max_len, nested_list):
  cnt = 0
  for s in nested_list:
    if(len(s) <= max_len):
        cnt = cnt + 1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (cnt / len(nested_list))*100))

In [41]:
max_len = 100
below_threshold_len(max_len, X_train)

전체 샘플 중 길이가 100 이하인 샘플의 비율: 91.72


## Train Model

- 추가적으로 layer를 더 쌓거나, 다른 hyperparameter를 조정해가면서 테스트를 합니다
  - (주의) 모델을 작성할때, binary가 아닌 multi-label이기 때문에, 최종 layer는 1이 아닌 label 수를 적어야 합니다.
- EarlyStopping 등 추가적인 내용도 함께 적용합니다.
- Multi-label classification이기 때문에 loss는 `categorical_crossentropy`를 적용하고, 마지막 layer의 activation function은 `softmax`로 합니다. ([참고자료](https://utto.tistory.com/8))


### (참고) Apply Attention Mechanism
 - [wikidocs 참고자료](https://wikidocs.net/48920)
 - [블로그 참고자료](https://hcnoh.github.io/2018-12-11-bahdanau-attention)
 - [논문](https://arxiv.org/pdf/1409.0473.pdf)

In [43]:
from tensorflow.keras.layers import Embedding, Dense, LSTM, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [44]:
vocab_size = 5000
tokenizer = Tokenizer(vocab_size, oov_token = 'OOV') 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [49]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()
y_train = np.array(train_df['label_encoded']).reshape(-1, 1)
y_test = np.array(test_df['label_encoded']).reshape(-1, 1)
encoder.fit(y_train)
y_train = encoder.transform(y_train).toarray()
y_test = encoder.transform(y_test).toarray()

In [50]:
model = Sequential([
                    Embedding(input_dim=vocab_size, output_dim=64),
                    # Bidirectional(LSTM(64)),
                    # batch size, timesteps, hidden state
                    LSTM(64, return_sequences=True), # output dimension size를 2가 아닌 3으로 하기 위해서
                    LSTM(64, return_sequences=True),
                    LSTM(64),
                    Dense(64),
                    Dense(64),
                    Dense(9, activation='softmax')
                    ])

In [51]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 64)          320000    
_________________________________________________________________
lstm_6 (LSTM)                (None, None, 64)          33024     
_________________________________________________________________
lstm_7 (LSTM)                (None, None, 64)          33024     
_________________________________________________________________
lstm_8 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_6 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_7 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_8 (Dense)              (None, 9)                

In [52]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint(filepath='best_model.ckpt', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

- batch_size, epochs 등 다른 parameter도 변경해서 실험해볼 수 있습니다.

In [54]:
X_train = np.asarray(X_train).astype(np.float32)
y_train = np.asarray(y_train).astype(np.float32)

/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


ValueError: ignored

In [55]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=5, callbacks=[es, mc], batch_size=60, validation_split=0.2)

ValueError: ignored

## Evaluate model

In [ ]:
print("Test Set에 대한 Accuracy는 다음과 같습니다.")
model.evaluate(X_test, y_test)

In [ ]:
predict_prob = model.predict(X_test)
predict_labels = np.argmax(predict_prob, axis = 1)
original_labels = np.argmax(y_test, axis = 1)

In [ ]:
test_df['preprocessed_sent'].head()

In [ ]:
for i in range(10):
  print(f"기사내용: {test_df['preprocessed_sent'].values[i]}")
  print(f"예측: {predict_labels[i]}, 정답: {original_labels[i]}")
  print("\n")

## Try random sentences

- '정치': sentence = "정부는 2012년 예산의 공고안과 배정계획을 1월3일 국무회의에서 의결하고 연초부터 바로 집행에 들어간다. 세계 경제의 불확실성이 높아지고 경기가 둔화할 가능성이 높은 만큼 조기 집행에 박차를 가할 예정이다"

- '경제': sentence = "국세청은 특히 서민생활에 피해를 주면서 폭리를 취하는 매점매석 농수산물 유통업체 등에 대한 추적조사를 강화하고, 지방청에 ‘민생침해 사업자 조사전담팀’을 꾸려 민생침해 탈세자에 대한 엄정 대응에 나설 계획이라고 밝혔다"

- '사회': sentence = "70대 운전사가 몰던 25인승 어린이 통학버스가 주유소로 돌진해 차량 3대를 들이 받았다. 다행히 통학버스에 운전자 외에는 탑승자가 없어 큰 인명피해는 피했다. 운전자는 차량 결함을 주장하고 있으나, 경찰은 운전자 과실 여부도 조사 중이다."

- '생활': sentence = "차진 식감과 부드러운 감촉을 모두 지닌 식빵, 결결이 찢어지는 크루아상, 둥글고 크게 구운 호밀빵 모두 모양새부터 알차고 단단했다. 디저트로 눈을 옮기면 국가 대표팀같이 뭐 하나 빼놓을 수 없는 케이크가 나란히 줄을 서 있었다."

- '연예': sentence = "공연은 말 그대로 다채로움 그 자체였다. 발레극인지, 현대무용극인지, 전통극인지, 연극인지, 연극이면 다인극인지 1인극인지 모를 정도로 다양한 장르의 결합이 먼저 눈에 띈다."
